In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_squared_error

# Learn to import modules in the usage order 😊 this will help in understand library structures


In [9]:
# Create a simple dataset

data = {

    "Size(sqft)" : [2000, 1500, 1800, 1200, 2500],

    "Bedrooms" : [3, 2, 3, 2, 4],  

    "Bathrooms" : [2, 1, 2, 1, 3],
      
    "Location Score" : [8, 7, 9, 6, 10],
     
    "Price ($)": [300000, 200000, 250000, 180000, 500000] }


In [10]:
# Hyperparameter Grid

param_grid = {

    'alpha' : [0.01, 0.1, 1.0, 10.0], # Regularization strenth
    'l1_ratio' : [0.2, 0.4, 0.6, 0.8, 1.0] # Mixing ratio
    }

# Initialize ElasticNet Model
elastic_net = ElasticNet(max_iter=1000, random_state=42)

# Set up GridSearchCV

grid_search = GridSearchCV(
    estimator=elastic_net,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use negative MSE for Evaluation
    cv = 4,  # 4-fold Cross-validation
    n_jobs= -1, # Use all CPU cores
    verbose=1
)



In [11]:
# Convert to dataframe

df = pd.DataFrame(data)

# Seperate Features and target

X = df[["Size(sqft)", "Bedrooms", "Bathrooms", "Location Score"]]

y = df[["Price ($)"]]

# Split the dataset into training and  testing

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2 , random_state = 42 )


# Standardize the features

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [19]:
# Perform the Grid search
grid_search.fit(X_train_scaled,y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(" \nBest parameters :", best_params ,'\n' )

Fitting 4 folds for each of 20 candidates, totalling 80 fits
 
Best parameters : {'alpha': 1.0, 'l1_ratio': 0.4} 



/opt/anaconda3/envs/masterxml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.909e+08, tolerance: 5.227e+06
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/masterxml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.462e+08, tolerance: 5.660e+06
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/masterxml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

In [13]:
# Train the final model with the best parameters
best_model = grid_search.best_estimator_

In [14]:
# Evaluate on the test set
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test,y_pred)

print("Test set Mean Squared Error :",mse)

Test set Mean Squared Error : 32338679.615366265


In [16]:
# Display final model coefficients
print(" \nElasticNet Coefficients:", best_model.coef_)
print("ElasticNet Intercept:", best_model.intercept_)

 
ElasticNet Coefficients: [28964.02462945 28530.64033804 28530.63935759 10846.32830622]
ElasticNet Intercept: [307500.]
